# Imports

In [2]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
from statsmodels.formula.api import ols
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns
import math
import random
from sklearn.model_selection import train_test_split

# Choix du projet

In [3]:
#Choisir le projet
project = 'AUQB'

# Récupération des données ERA5

In [4]:
S3_bucket = 'edfred-edfre-sbx-eu-west-1-solar-radiation-data'
S3_era5_folder = r'EtudeWindIndex/ERA5'

In [5]:
#Lecture de l'export horraire de ERA5
file = 'ERA5_' + project
S3_era5 = f's3://{S3_bucket}/{S3_era5_folder}/{file}.csv'
era5 = pd.read_csv(S3_era5, sep=';')

In [6]:
#Visualisation
era5.sample(5)

,time,d2m,t2m,sp,ws100,wd100,ws10,wd10,E100,rh,density,E100_cor
121677,2013-11-17 21:00:00,282.66150,287.81192,101713.920,13.252587,38.0,11.230343,38.0,2869.331188,74.247894,1.231124,2883.676636
185513,2021-02-28 17:00:00,279.49265,287.29272,102700.086,12.655601,45.0,9.761226,47.0,2736.091196,60.999603,1.245307,2781.448379
182724,2020-11-04 12:00:00,288.13900,290.41895,102149.680,4.387108,97.0,4.018655,97.0,120.743240,88.600310,1.225300,120.772803
54857,2006-04-04 17:00:00,284.79883,286.61690,101229.414,2.592979,131.0,2.173413,129.0,0.000000,90.909580,1.230369,0.000000
159340,2018-03-06 04:00:00,279.59058,282.92548,99061.734,9.529609,48.0,8.483870,48.0,1504.439095,83.325500,1.219731,1497.968702


In [7]:
#On converti le timestamp
era5['year'] = era5.time.map(lambda date: int(date[:4]))
era5['month'] = era5.time.map(lambda date: int(date[5:7]))
era5['day'] = era5.time.map(lambda date: int(date[8:10]))
era5['hour'] = era5.time.map(lambda date: int(date[11:13]))

In [8]:
#On garde les informations utiles
era5.drop(['time','d2m','t2m','sp','wd100','ws10','wd10','rh','density','E100_cor'], axis=1, inplace=True)
era5.rename(columns={"E100": "energy", "ws100":"windspeed_era5"}, inplace=True)
era5 = era5[['year','month','day','hour','energy','windspeed_era5']]

In [9]:
#Visualisation
era5.sample(5)

,year,month,day,hour,energy,windspeed_era5
165011,2018,10,28,11,2868.483264,13.245963
44950,2005,2,15,22,2892.821575,13.436106
191108,2021,10,19,20,294.761943,5.642711
146514,2016,9,17,18,1522.601396,9.571170
73637,2008,5,26,5,2994.773106,14.992437


# Récupération des donées 10 min

In [10]:
#Lecture de l'export horraire turbine
S3_bucket = 'edfred-edfre-sbx-eu-west-1-solar-radiation-data'
S3_real_folder = r'EtudeWindIndex/Real'
file = '10min_' + project
S3_real = f's3://{S3_bucket}/{S3_real_folder}/{file}.csv'
real_set = pd.read_csv(S3_real, sep=';')

In [11]:
#Visualisation
real_set.sample(5)

,project,turbine,year,month,day,hour,minute,active_power_avg,wind_speed_avg
245586,AUQB,8,2020,12,7,2,10,161.0,5.6
4224272,AUQB,8,2021,6,23,1,0,855.0,9.3
3215175,AUQB,6,2021,2,19,12,30,674.0,8.8
4621886,AUQB,7,2021,11,25,6,50,1190.0,10.4
1942232,AUQB,10,2020,7,19,13,0,NaN,NaN


# Création des trainset et testset

In [12]:
Dic_turbines = {}

for num in list_num_turbine :
    
    pd.options.mode.chained_assignment = None
    
    tts = real_set[real_set.turbine==num]
    trainset, testset = train_test_split(tts, test_size=0.33, random_state=42, stratify=tts.loc[:,['year','month','day','hour']])
    trainset.sort_values(by=['year','month','day','hour'], inplace=True)
    testset.sort_values(by=['year','month','day','hour'], inplace=True)
    
    key_train  = 'trainset'+str(num)
    key_test = 'testset'+str(num)
    Dic_turbines[key_train] = trainset
    Dic_turbines[key_test] = testset

In [13]:
#Visualisation
Dic_turbines[random.choice(['trainset','testset'])+str(random.choice(list_num_turbine))].sample(5)

,project,turbine,year,month,day,hour,minute,active_power_avg,wind_speed_avg
1614017,AUQB,7,2020,11,19,16,30,1845.0,15.3
2129866,AUQB,7,2021,4,6,3,30,1156.0,10.3
114365,AUQB,7,2020,6,7,20,10,1729.0,12.9
2012226,AUQB,7,2020,10,24,18,10,-2.0,3.0
4711989,AUQB,7,2021,10,28,10,10,397.0,7.2


# Synthèse des données utiles

In [14]:
#On choisit une turbine
num_turbine = random.choice(list_num_turbine)

In [15]:
key_train  = 'trainset'+str(num_turbine)
key_test = 'testset'+str(num_turbine)
trainset_turbine = Dic_turbines[key_train]
testset_turbine = Dic_turbines[key_test]

In [16]:
#On fusionne les données météo et de production
train = pd.merge(era5.loc[era5.year==2021], trainset_turbine, how='left', on=['year','month','day','hour'])

In [17]:
#Filtrage des données manquantes ou éronnées *
train = train[~((train.wind_speed_avg.isnull()) | (train.wind_speed_avg == 0))]
#Visualisation
train.sample(5)

,year,month,day,hour,energy,windspeed_era5,project,turbine,minute,active_power_avg,wind_speed_avg
387964,2021,10,28,3,1863.075882,10.351899,AUQB,9,0,0.0,5.0
322914,2021,9,18,21,1146.586557,8.673350,AUQB,9,20,1680.0,12.6
396965,2021,11,2,14,3000.000000,18.345582,AUQB,9,0,73.0,4.7
230903,2021,7,22,7,0.000000,2.117977,AUQB,9,40,168.0,5.7
42544,2021,2,17,10,174.322940,4.861265,AUQB,9,20,51.0,4.1


In [18]:
#On fusionne les données météo et de production
test = pd.merge(era5.loc[era5.year==2021], testset_turbine, how='left', on=['year','month','day','hour'])

In [19]:
#Filtrage des données manquantes ou éronnées *
test = test[~((test.wind_speed_avg.isnull()) | (test.wind_speed_avg == 0))]
#Visualisation
test.sample(5)

,year,month,day,hour,energy,windspeed_era5,project,turbine,minute,active_power_avg,wind_speed_avg
129737,2021,8,11,10,0.000000,1.776283,AUQB,9,40,0.0,3.1
212659,2021,11,25,9,2995.163599,15.032720,AUQB,9,50,306.0,6.2
209768,2021,11,21,17,442.966454,6.394590,AUQB,9,40,233.0,6.1
5360,2021,1,13,9,0.000000,2.607426,AUQB,9,0,1937.0,14.5
117200,2021,7,25,6,0.000000,2.837737,AUQB,9,50,0.0,7.1


# Construction du dataframe de régression horraire

In [ ]:
#Calcul des valeurs horaires *
train_hourly = pd.DataFrame(columns=['year','month','day','hour','energy','active_power','windspeed_data','windspeed_era5'])
for year in train.year.unique().tolist() :
    for month in train[train.year==year].month.unique().tolist() :
        for day in train[(train.year==year)&(train.month==month)].day.unique().tolist() :
            for hour in train[(train.year==year)&(train.month==month)&(train.day==day)].hour.unique().tolist() :
                
                mask_data = (train.year==year) & (train.month==month) & (train.day==day) & (train.hour==hour)
                windspeed_data = train.loc[mask_data].wind_speed_avg.unique().mean()
                active_power = train.loc[mask_data].active_power_avg.unique().mean()
                
                mask_era5 = (era5.year==year) & (era5.month==month) & (era5.day==day) & (era5.hour==hour)
                try :
                    energy = era5.loc[mask_era5].energy.iloc[0]
                    windspeed_era5 = float(era5.loc[mask_era5].windspeed_era5)
                except :
                    energy = 0
                    windspeed_era5 = 0
                
                train_hourly = train_hourly.append({'year':year,'month':month,'day':day,'hour':hour,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#Filtrage des données manquantes ou éronnées *
train_hourly = train_hourly[~((train_hourly.windspeed_data.isnull())|(train_hourly.windspeed_era5 == 0))]
#Visualisation
train_hourly.sample(5)

In [ ]:
#Filtrage des données manquantes ou éronnées *
train_hourly = train_hourly[~((train_hourly.active_power.isnull())|(train_hourly.active_power == 0))]
train_hourly = train_hourly[~((train_hourly.windspeed_data.isnull())|(train_hourly.windspeed_data == 0))]
train_hourly = train_hourly[~((train_hourly.windspeed_era5.isnull())|(train_hourly.windspeed_era5 == 0))]
train_hourly = train_hourly[~((train_hourly.energy.isnull())|(train_hourly.energy == 0))]

#Visualisation
train_hourly.sample(5)

In [ ]:
#Calcul de l'écart entre les données de vent météo et mesurées
lim=1 #écart > lim
sum(abs(train_hourly.windspeed_data - train_hourly.windspeed_era5)>lim)/len(train_hourly)

In [ ]:
#Calcul des valeurs horaires *
test_hourly = pd.DataFrame(columns=['year','month','day','hour','energy','active_power','windspeed_data','windspeed_era5'])
for year in test.year.unique().tolist() :
    for month in test.month.unique().tolist() :
        for day in test.day.unique().tolist() :
            for hour in test.hour.unique().tolist() :
                
                mask_data = (test.year==year) & (test.month==month) & (test.day==day) & (test.hour==hour)
                windspeed_data = test.loc[mask_data].wind_speed_avg.mean()
                active_power = test.loc[mask_data].active_power_avg.mean()
                
                mask_era5 = (era5.year==year) & (era5.month==month) & (era5.day==day) & (era5.hour==hour)
                try :
                    energy = era5.loc[mask_era5].energy.iloc[0]
                    windspeed_era5 = float(era5.loc[mask_era5].windspeed_era5)
                except :
                    energy = 0
                    windspeed_era5 = 0
                
                test_hourly = test_hourly.append({'year':year,'month':month,'day':day,'hour':hour,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#Filtrage des données manquantes ou éronnées *
test_hourly = test_hourly[~((test_hourly.windspeed_data.isnull())|(test_hourly.windspeed_era5 == 0))]
#Visualisation
test_hourly.sample(5)

In [ ]:
#Calcul de l'écart entre les données de vent météo et mesurées
lim=1 #écart > lim
sum(abs(test_hourly.windspeed_data - test_hourly.windspeed_era5)>lim)/len(test_hourly)

# Régression

In [ ]:
#définition du calcul de l'erreur (root minimal square error)
def rmse_calc(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
plots = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Models/Model_6/plots_m6.pdf')

In [ ]:
params = []

with PdfPages(plots) as pdf:
    #On récupère le vecteur de données
    X = train_hourly
        
    #Calcul des outliers
    m = ols('active_power ~ energy',X).fit()
    infl = m.get_influence()
    sm_fr = infl.summary_frame()
    cooks_d_tresh = 4 / len(X)
    mask = sm_fr.cooks_d < cooks_d_tresh
    #Régression sans les outliers
    m_clean = ols('active_power ~ energy', X.loc[mask,]).fit()

    #On récupère les paramètres à afficher
    rmse = round(rmse_calc(m_clean.predict(X.loc[mask,"energy"]),X.loc[mask,"active_power"]),0)
    intercept = round(m_clean.params.Intercept,0)
    slope = round(m_clean.params.energy,2)
    rsquare = round(m_clean.rsquared, 2)
    #On sauvegarde les paramètres
    params.append([project, slope, intercept, rmse, rsquare])
        
    #On trace le graphique
    fig = plt.figure(figsize=(12, 5))
    plt.suptitle(project,fontsize=16, fontweight="bold")
    plt.subplot(1, 2, 1)
    plt.title(f" R²={rsquare}\n rmse = {round(rmse/1000,0)} MWh\n  y = {slope} x + {intercept}", size=12, fontweight="bold")
    sns.scatterplot(x='energy', y='active_power',data=X.loc[mask,]) 
    sns.scatterplot(x='energy', y='active_power',data=X.loc[~mask,], color='r')
    sns.lineplot(x=X.energy, y=m.predict(X.energy), color='r', alpha=0.3)
    sns.lineplot(x=X.energy, y=m_clean.predict(X.energy))
    plt.xticks(color='w')
    plt.yticks(color='w')
    #Turn off tick labels
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    #On sauvegarde le graphique
    pdf.savefig(fig)

    plt.close()

# Regression journalière

In [ ]:
#On crée un trainset journalier
train_daily = pd.DataFrame(columns=['year','month','day','energy','active_power', 'windspeed_data','windspeed_era5'])
for year in train_hourly.year.unique().tolist() :
    for month in train_hourly.month.unique().tolist() :
        for day in train_hourly.day.unique().tolist() :
            mask = (train_hourly.year==year) & (train_hourly.month==month) & (train_hourly.day==day)
            energy = train_hourly.loc[mask].energy.mean()
            active_power = train_hourly.loc[mask].active_power.mean()
            windspeed_data = train_hourly.loc[mask].windspeed_data.mean()
            windspeed_era5 = train_hourly.loc[mask].windspeed_era5.mean()
            train_daily = train_daily.append({'year':year,'month':month,'day':day,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#On  converti dans les dates en entiers
train_daily.year = train_daily.year.apply(int)
train_daily.month = train_daily.month.apply(int)
train_daily.day = train_daily.day.apply(int)

In [ ]:
#Visualisation
train_daily.dropna(inplace=True)
train_daily.sample(5)

In [ ]:
test = test.reset_index()

In [ ]:
#On crée un trainset journalier
test_daily = pd.DataFrame(columns=['year','month','day','energy','active_power', 'windspeed_data','windspeed_era5'])
for year in test_hourly.year.unique().tolist() :
    for month in test_hourly.month.unique().tolist() :
        for day in test_hourly.day.unique().tolist() :
            mask = (test_hourly.year==year) & (test_hourly.month==month) & (test_hourly.day==day)
            energy = test_hourly.loc[mask].energy.mean()
            active_power = test_hourly.loc[mask].active_power.mean()
            windspeed_data = test_hourly.loc[mask].windspeed_data.mean()
            windspeed_era5 = test_hourly.loc[mask].windspeed_era5.mean()
            test_daily = test_daily.append({'year':year,'month':month,'day':day,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#On  converti dans les dates en entiers
test_daily.year = test_daily.year.apply(int)
test_daily.month = test_daily.month.apply(int)
test_daily.day = test_daily.day.apply(int)

In [ ]:
#Visualisation
test_daily.dropna(inplace=True)
test_daily.sample(5)

In [ ]:
plot_daily = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Models/Model_6/plots_day_m6.pdf')

In [ ]:
params = []

with PdfPages(plot_daily) as pdf:
    #On récupère le vecteur de données
    X = train_daily
        
    #Calcul des outliers
    m = ols('active_power ~ energy',X).fit()
    infl = m.get_influence()
    sm_fr = infl.summary_frame()
    cooks_d_tresh = 4 / len(X)
    mask = sm_fr.cooks_d < cooks_d_tresh
    #Régression sans les outliers
    m_clean = ols('active_power ~ energy', X.loc[mask,]).fit()

    #On récupère les paramètres à afficher
    rmse = round(rmse_calc(m_clean.predict(X.loc[mask,"energy"]),X.loc[mask,"active_power"]),0)
    intercept = round(m_clean.params.Intercept,0)
    slope = round(m_clean.params.energy,2)
    rsquare = round(m_clean.rsquared, 2)
    #On sauvegarde les paramètres
    params.append([project, slope, intercept, rmse, rsquare])
        
    #On trace le graphique
    fig = plt.figure(figsize=(12, 5))
    plt.suptitle(project,fontsize=16, fontweight="bold")
    plt.subplot(1, 2, 1)
    plt.title(f" R²={rsquare}\n rmse = {round(rmse/1000,0)} MWh\n  y = {slope} x + {intercept}", size=12, fontweight="bold")
    sns.scatterplot(x='energy', y='active_power',data=X.loc[mask,]) 
    sns.scatterplot(x='energy', y='active_power',data=X.loc[~mask,], color='r')
    sns.lineplot(x=X.energy, y=m.predict(X.energy), color='r', alpha=0.3)
    sns.lineplot(x=X.energy, y=m_clean.predict(X.energy))
    plt.xticks(color='w')
    plt.yticks(color='w')
    #Turn off tick labels
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    #On sauvegarde le graphique
    pdf.savefig(fig)

    plt.close()

# Régression mensuelle

In [ ]:
#On crée un trainset journalier
train_monthly = pd.DataFrame(columns=['year','month','energy','active_power', 'windspeed_data','windspeed_era5'])
for year in train_hourly.year.unique().tolist() :
    for month in train_hourly.month.unique().tolist() :
        for day in train_hourly.day.unique().tolist() :
            mask = (train_hourly.year==year) & (train_hourly.month==month)
            energy = train_hourly.loc[mask].energy.mean()
            active_power = train_hourly.loc[mask].active_power.mean()
            windspeed_data = train_hourly.loc[mask].windspeed_data.mean()
            windspeed_era5 = train_hourly.loc[mask].windspeed_era5.mean()
            train_monthly = train_monthly.append({'year':year,'month':month,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#On  converti dans les dates en entiers
train_monthly.year = train_monthly.year.apply(int)
train_monthly.month = train_monthly.month.apply(int)

In [ ]:
#Visualisation
train_monthly.dropna(inplace=True)
train_monthly.sample(5)

In [ ]:
#On crée un testset mensuel
test_monthly = pd.DataFrame(columns=['year','month','active_power','energy', 'windspeed_data','windspeed_era5'])
for year in test_hourly.year.unique().tolist() :
    for month in test_hourly.month.unique().tolist() :
        for day in test_hourly.day.unique().tolist() :
            mask = (test_hourly.year==year) & (test_hourly.month==month) & (test_hourly.day==day)
            energy = test_hourly.loc[mask].energy.mean()
            active_power = test_hourly.loc[mask].active_power.mean()
            windspeed_data = test_hourly.loc[mask].windspeed_data.mean()
            windspeed_era5 = test_hourly.loc[mask].windspeed_era5.mean()
            test_monthly = test_monthly.append({'year':year,'month':month,'energy':energy,'active_power':active_power,'windspeed_data':windspeed_data,'windspeed_era5':windspeed_era5},ignore_index=True)

In [ ]:
#On  converti dans les dates en entiers
test_monthly.year = test_monthly.year.apply(int)
test_monthly.month = test_monthly.month.apply(int)

In [ ]:
#Visualisation
test_monthly.dropna(inplace=True)
test_monthly.sample(5)

In [ ]:
plot_monthly = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Models/Model_6/plots_month_m6.pdf')

In [ ]:
params = []

with PdfPages(plot_monthly) as pdf:
    #On récupère le vecteur de données
    X = train_monthly
        
    #Calcul des outliers
    m = ols('active_power ~ energy',X).fit()
    infl = m.get_influence()
    sm_fr = infl.summary_frame()
    cooks_d_tresh = 4 / len(X)
    mask = sm_fr.cooks_d < cooks_d_tresh
    #Régression sans les outliers
    m_clean = ols('active_power ~ energy', X.loc[mask,]).fit()

    #On récupère les paramètres à afficher
    rmse = round(rmse_calc(m_clean.predict(X.loc[mask,"energy"]),X.loc[mask,"active_power"]),0)
    intercept = round(m_clean.params.Intercept,0)
    slope = round(m_clean.params.energy,2)
    rsquare = round(m_clean.rsquared, 2)
    #On sauvegarde les paramètres
    params.append([project, slope, intercept, rmse, rsquare])
        
    #On trace le graphique
    fig = plt.figure(figsize=(12, 5))
    plt.suptitle(project,fontsize=16, fontweight="bold")
    plt.subplot(1, 2, 1)
    plt.title(f" R²={rsquare}\n rmse = {round(rmse/1000,0)} MWh\n  y = {slope} x + {intercept}", size=12, fontweight="bold")
    sns.scatterplot(x='energy', y='active_power',data=X.loc[mask,]) 
    sns.scatterplot(x='energy', y='active_power',data=X.loc[~mask,], color='r')
    sns.lineplot(x=X.energy, y=m.predict(X.energy), color='r', alpha=0.3)
    sns.lineplot(x=X.energy, y=m_clean.predict(X.energy))
    plt.xticks(color='w')
    plt.yticks(color='w')
    #Turn off tick labels
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    #On sauvegarde le graphique
    pdf.savefig(fig)

    plt.close()